# A `value_from:` Policy

In [2]:
policy_text = """
name: asg-invalid-asv-value-mark
resource: asg
comment: 'Report on any ASGs that use an ASV that isn''t valid.

  '
filters:
- tag:custodian_asv: absent
- key: CreatedTime
  op: greater-than
  type: value
  value: 0.011
  value_type: age
- key: tag:ASV
  op: not-in
  type: value
  value_from:
    expr: all_values.*
    format: json
    url: s3://fs-cloud-governance/custodian/asv_list.json
actions:
- tag: custodian_asv
  type: tag
  value: AutoScaling Group does not have a valid ASV tag value
- action_desc: The ASV tag should be updated, both on the ASG and in any template
    or config files.
  cc:
  - FSCustodianNotifications@capitalone.com
  from: FSCustodianNotifications@capitalone.com
  policy_url: https://pulse.kdc.capitalone.com/docs/DOC-138928
  subject: '[custodian][{{ account }}] AutoScaling Group with invalid ASV value -
    {{ region }}'
  template: fs-default.html
  to:
  - resource-owner
  transport:
    topic: arn:aws:sns:{region}:123456789012:c7n-notifications
    type: sns
  type: notify
  violation_desc: 'The following AutoScaling Group(s) are tagged with an invalid ASV
    value:'
"""

In [86]:
from xlate.c7n_to_cel import C7N_Rewriter
cel_text = C7N_Rewriter.c7n_rewrite(policy_text)
print(cel_text)

absent(Resource["Tags"].filter(x, x["Key"] == "custodian_asv")[0]["Value"]) && Now - duration("15m50s") > timestamp(Resource["CreatedTime"]) && ! value_from("s3://fs-cloud-governance/custodian/asv_list.json", "json").jmes_path('all_values.*').contains(Resource["Tags"].filter(x, x["Key"] == "ASV")[0]["Value"])


Reformatting:

    absent(Resource["Tags"].filter(x, x["Key"] == "custodian_asv")[0]["Value"]) 
    && Now - duration("15m50s") > timestamp(Resource["CreatedTime"]) 
    && ! value_from("s3://fs-cloud-governance/custodian/asv_list.json", "json")
        .jmes_path('all_values.*').contains(
            Resource["Tags"].filter(x, x["Key"] == "ASV")[0]["Value"]
        )

How can we test with `value_from`?? Do we really need access to an S3? 

It turns out, we can provide our own extension function that provides appropriate test data. We can replace the `value_from()` function.

In [93]:
import celpy.celtypes
from celpy.adapter import json_to_cel

def my_value_from(url, format):
    print(f"Reading {url} in {format}")
    the_document = {"all_values": {"name": "in_the_list"}}
    return json_to_cel(the_document)

We can update `c7nlib.FUNCTIONS` with the new function.

In [94]:
import celpy.c7nlib
my_functions = celpy.c7nlib.FUNCTIONS
my_functions["value_from"] = my_value_from

Now, we can 

In [95]:
import celpy
decls = {
    "Resource": celpy.celtypes.MapType,
    "Now": celpy.celtypes.TimestampType,
}
env = celpy.Environment(annotations=decls)
ast = env.compile(cel_text)
# Bind my_functions with the CEL code...
prgm = env.program(ast, functions=my_functions)

In [96]:
sample_doc_in = {
    "Tags": [
        {"Key": "ASV", "Value": "in_the_list"},
    ],
    "CreatedTime": "2020-10-17T18:00:00Z"
}
activation_in = {
    "Resource": celpy.adapter.json_to_cel(sample_doc_in),
    "Now": celpy.celtypes.TimestampType("2020-10-17T18:19:20Z"),
}
result = prgm.evaluate(activation_in)
result

Reading s3://fs-cloud-governance/custodian/asv_list.json in json


BoolType(False)

In [97]:
sample_doc_not_in = {
    "Tags": [
        {"Key": "ASV", "Value": "not_in_the_list"},
    ],
    "CreatedTime": "2020-10-17T18:00:00Z"
}
activation_in = {
    "Resource": celpy.adapter.json_to_cel(sample_doc_in),
    "Now": celpy.celtypes.TimestampType("2020-10-17T18:19:20Z"),
}
result = prgm.evaluate(activation_in)
result

Reading s3://fs-cloud-governance/custodian/asv_list.json in json


BoolType(False)

## Debugging Approach

Breaking the CEL (or the resource description) into pieces, and evaluating each piece.

In [98]:
jmes_path_cel_1 = """
value_from("s3://fs-cloud-governance/custodian/asv_list.json", "json")
"""
ast = env.compile(jmes_path_cel_1)
# Bind my functions with the CEL code
my_functions = celpy.c7nlib.FUNCTIONS
my_functions["value_from"] = my_value_from
prgm = env.program(ast, functions=my_functions)
result = prgm.evaluate(context={})
result

Reading s3://fs-cloud-governance/custodian/asv_list.json in json


MapType({StringType('all_values'): MapType({StringType('name'): StringType('in_the_list')})})

In [99]:
jmes_path_cel_2 = """
value_from("s3://fs-cloud-governance/custodian/asv_list.json", "json")
.jmes_path('all_values.*')
"""
ast = env.compile(jmes_path_cel_2)
# Bind my functions with the CEL code
my_functions = celpy.c7nlib.FUNCTIONS
my_functions["value_from"] = my_value_from
prgm = env.program(ast, functions=my_functions)
result = prgm.evaluate(context={})
result

Reading s3://fs-cloud-governance/custodian/asv_list.json in json


ListType([StringType('in_the_list')])

In [100]:
jmes_path_cel_3 = """
value_from("s3://fs-cloud-governance/custodian/asv_list.json", "json")
.jmes_path('all_values.*').contains("in_the_list")
"""
ast = env.compile(jmes_path_cel_3)
# Bind my functions with the CEL code
my_functions = celpy.c7nlib.FUNCTIONS
my_functions["value_from"] = my_value_from
prgm = env.program(ast, functions=my_functions)
result = prgm.evaluate(context={})
result

Reading s3://fs-cloud-governance/custodian/asv_list.json in json


BoolType(True)

# Using `c7nlib` -- Future Feature

This will require C7N installed locally. We don't really need **all** of custodian, but it's hard to break it into pieces. So. Get the Cloud Custodian Rule Engine. 

Set your `PYTHONPATH` to include CEL **AND** C7N.

We will rely on a plug-in to C7N. This will expose some features so CEL expressions can make use of them.

We're going to write a `CELFilter` subclass of `Filter`. This will require refactoring several existing filters
to pull code into mixin class definitions that can be shared by legacy filters and `CELFilter`. 

The code *should* look something like this:

     class CELFilter(Filter):
        """
        State a filter as a CEL expression.

        The activation environment for CEL has all the functions in ``c7nlib``.

        It also includes three global objects:

        -   ``Resource`` the cloud resource to be examined.

        -   ``Now`` the current time as a CEL timestamp.

        -   ``Event`` an Event for those resources where the C7N check is triggered by a CloudWatch event.

        """

        schema = type_schema(
            'cel',
            'type': {'enum': ['cel']},
            'expr': {'type': 'string'}
        )

        def __init__(self, data, manager):
            super(Missing, self).__init__(data, manager)

        def get_instance_image(self, resource): pass

        def get_related_ids(self, resource): pass

        def get_related(self, resource): pass

        def get_credential_report(self): pass

        def get_matching_aliases(self): pass
        
There will be more. We're working on it.